In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time
import re
from selenium import webdriver
import numpy as np
from PIL import Image
from io import BytesIO
import requests
import os
import json
from urllib.parse import unquote
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementClickInterceptedException
from gmaps import extract_location_details_from_google_maps_url

In [4]:
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=chrome_options)
driver.maximize_window()

url_list=['naked-development',
'webgamma',
 'elevate-my-brand',
 'geeky-social-ltd',
 'online-optimism',
 'ewr-digital']
#  'manifesto',
#  'encyphers',
#  'grazitti-interactive',
#  'highervisibility',
#  'more-by-us',
#  'savas-labs',
#  '3eighty',
#  '2stallions',
#  'wyperformance',
#  'splendor',
#  'blublu-studios',
#  'evestar',
#  'seota-digital-marketing',
#  'admiral-media']

id_inc=8880000000
json_list=[]
for i in url_list:

    #Getting URL
    url='https://www.designrush.com/agency/profile/'+str(i) #data
    driver.get(url)

    #uid
    id_inc+=1
    uid=id_inc #data

    #title
    try:
        title=driver.find_element(By.CSS_SELECTOR,'h1.title').text
    except Exception:
        title=url.split('/')[-1]

    #review rating
    try:
        rating=driver.find_element(By.CLASS_NAME,'review-rating').text
    except NoSuchElementException:
        rating=''

    #Logo
    try:
        image_element = driver.find_element(By.CSS_SELECTOR,".profile-header--image img")
        image_source = image_element.get_attribute("src")
        response = requests.get(image_source)
        image = Image.open(BytesIO(response.content))
        logo_path=f'gallery//images//{uid}//logo//{title}.png'
        logo_path1=logo_path.replace("//",'/')
        directory = os.path.dirname(logo_path)
        if not os.path.exists(directory):
            os.makedirs(directory)
        with open(logo_path, "wb") as f:
            f.write(response.content)
    except NoSuchElementException:
        logo_path=""

    #Profile Slogan
    try:
        profile_slogan=driver.find_element(By.CLASS_NAME,'profile-header--slogan').text.strip()
    except NoSuchElementException:
        profile_slogan=""

    #full address
    try:
        full_address=driver.find_element(By.CLASS_NAME,'full-address').text
    except NoSuchElementException:
        full_address=''

    #contacts (Website url)
    try:
        contacts=driver.find_element(By.CSS_SELECTOR,'div.agency-site a').get_attribute('href')
    except NoSuchElementException:
        contacts=''
    
    #Social Media (facebook,twitter,linkedin,instagram,youtube)
    try:
        social_media=[b.get_attribute('href') for b in driver.find_elements(By.CSS_SELECTOR,"div.overview-socials--list a")]

        pattern_twiiter = re.compile(r'.*twitter.*')
        twitter = next((url for url in social_media if pattern_twiiter.search(url)), None)
        pattern_facebook = re.compile(r'.*facebook.*')
        facebook = next((url for url in social_media if pattern_facebook.search(url)), None)
        pattern_linkedin = re.compile(r'.*linkedin.*')
        linkedin = next((url for url in social_media if pattern_linkedin.search(url)), None)
        pattern_yt = re.compile(r'.*youtube.*')
        youtube = next((url for url in social_media if pattern_yt.search(url)), None)
        pattern_ig = re.compile(r'.*instagram.*')
        instagram = next((url for url in social_media if pattern_ig.search(url)), None)

        twitter = "" if twitter is None else twitter
        facebook = "" if facebook is None else facebook
        linkedin = "" if linkedin is None else linkedin
        youtube = "" if youtube is None else youtube
        instagram = "" if instagram is None else instagram    

    except NoSuchElementException:
        twiiter=''
        facebook=''
        linkedin=''
        youtube=''
        instagram=''
        pass

    #Number of employees|Minimal Budget|Average Hourly Rate|Year Founded
    try:
        try:
            overview_adds_text=driver.find_elements(By.CLASS_NAME,'overview-adds--text')
            overview_adds_text_list=[i.text for i in overview_adds_text]
            number_of_employees=overview_adds_text_list[0]
            minimal_budget=overview_adds_text_list[1].replace("$",'').replace(" ",'').replace(",",'')
            average_hourly_rate=overview_adds_text_list[2].replace('/hr','').replace("$",'')
            year_founded=overview_adds_text_list[3]
        except IndexError:
            number_of_employees=""
            minimal_budget=""
            average_hourly_rate=''
            year_founded=''
    except NoSuchElementException:
        number_of_employees=""
        minimal_budget=""
        average_hourly_rate=''
        year_founded='' 

    #Content
    try:
        content=driver.find_element(By.CLASS_NAME,'profile-overview--text').text.replace('\n',' ').strip()
    except NoSuchElementException:
        content=""

    #Headquarters
    try:
        headquarters=driver.find_element(By.CSS_SELECTOR,'div.full-address--data').text
    except NoSuchElementException:
        headquarters=""
        
    #Headquarters MapView
    try:
        headquarters_map=driver.find_element(By.CSS_SELECTOR,'.full-address--view a').get_attribute('href')
    except NoSuchElementException:
        headquarters_map=""
    
    #Country State City
    try:
        hq=extract_location_details_from_google_maps_url(headquarters_map)
        area=hq['area'].strip()
        city=hq['city'].strip()
        state=hq['postcode'].strip()
        country=hq['country'].strip()
    except Exception as e:
        address = headquarters_map.split("/search/")[-1]
        decoded_address = unquote(address)
        address_parts = list(filter(None, decoded_address.split(',')))
        country = address_parts[-1].strip().replace("+"," ")
        state = address_parts[-2].strip().replace("+"," ")
        city = address_parts[-3].strip().replace("+"," ")
        area=''

    #Other Locations
    try:
        other_locations_list=[j.text for j in driver.find_elements(By.CSS_SELECTOR,'div.full-address--data')]
        other_locations_map_list=[j.get_attribute('href') for j in driver.find_elements(By.CSS_SELECTOR,'.full-address--view a')]

        if len(other_locations_list)>1:
            other_locations=other_locations_list[1:]
            other_locations_map=other_locations_map_list[1:]
            other_loc_json_list=[]
            for k,l in zip(other_locations,other_locations_map):

                hq=extract_location_details_from_google_maps_url(l)
                area=hq['area'].strip()
                city=hq['city'].strip()
                state=hq['postcode'].strip()
                country=hq['country'].strip()

                other_locations_dict={
                    'other-location-address':k,
                    'other-location-mapview':l,
                    'country':country,
                    'state':state,
                    'city':city,
                    'area':area,
                }
                other_loc_json_list.append(other_locations_dict)
            other_loc_json = json.dumps(other_loc_json_list, indent=4)
            other_loc_json = json.loads(other_loc_json)
        else:
            other_loc_json=''
    except NoSuchElementException:
        other_loc_json=''
        pass

    #Video links
    try:
        gallery=driver.find_element(By.CSS_SELECTOR,'.profile-carousel').get_attribute('data-photos-and-videos')
        data_list = eval(gallery)
        video_urls = [item['videoUrl'] for item in data_list]
        video_urls=[url.replace("\\/", "/") for url in video_urls]
        video_urls=list(filter(lambda x:x!='',video_urls))
        if len(video_urls)==0:
            video_urls=""            
    except NoSuchElementException:
        video_urls=''

    #Image  Links
    try:
        thumbnail_image_urls = driver.find_elements(By.CSS_SELECTOR,".profile-carousel--item.thumbnail-image img")
        thumbnail_image_urls_list=[i.get_attribute('src') for i in thumbnail_image_urls]
        pg=0
        thumbnail_saved_path=[]
        for i in thumbnail_image_urls_list:
            response = requests.get(i)
            image = Image.open(BytesIO(response.content))
            pg+=1
            save_path=f'gallery//images//{uid}//thumbnail//{title}_thumbnail_pic_{pg}.png'
            thumbnail_saved_path.append(save_path)
            directory = os.path.dirname(save_path)
            if not os.path.exists(directory):
                os.makedirs(directory)
            with open(save_path, "wb") as f:
                f.write(response.content)
        if len(thumbnail_saved_path)==0:
            thumbnail_saved_path=""
        thumbnail_saved_path1=[k.replace("//",'/') for k in thumbnail_saved_path]
    except NoSuchElementException:
        thumbnail_saved_path=''

    #Services
    try:
        services=driver.find_elements(By.CLASS_NAME,'services')
        services_list=[i.text for i in services]
        services_list=services_list[0].split('\n')
        service_list_list=[]
        for k in services_list:
            service_dict={
                'service-type':k,
                'service-percent':'0%'
            }
            service_list_list.append(service_dict)

    except NoSuchElementException:
        services_list=''
        pass

    #Team Bios
    try:
        team_bio_name=[k.text for k in driver.find_elements(By.CLASS_NAME,'tab-teambio--member-name')]
        team_bio_title=[k.text for k in driver.find_elements(By.CLASS_NAME,'tab-teambio--title')]
        team_bio_des=[k.text for k in driver.find_elements(By.CLASS_NAME,'tab-teambio--description')]
        team_bio_image=[k.get_attribute('src') for k in driver.find_elements(By.CSS_SELECTOR,'div.tab-teambio--image img')]
        team_bio_json_list=[]
        pgg=0
        for t,g,h,l in zip(team_bio_name,team_bio_title,team_bio_des,team_bio_image):
            pgg+=1
            response = requests.get(l)
            image = Image.open(BytesIO(response.content))
            team_bio_image_path=f'gallery//images//{uid}//team_bios//{title}_team_bio_pic_{pgg}.png'
            team_bio_image_path1=team_bio_image_path.replace("//",'/')
            directory = os.path.dirname(team_bio_image_path)
            if not os.path.exists(directory):
                os.makedirs(directory)
            with open(team_bio_image_path, "wb") as f:
                f.write(response.content)

            team_bio_dict={
                'team-bio-name':t,
                'team-bio-title':g,
                'team-bio-description':h,
                'team-bio-image':team_bio_image_path1
            }
            team_bio_json_list.append(team_bio_dict)
    except NoSuchElementException:
        team_bio_json_list=''


    #Client profile
    try:
        clients=[k.text for k in driver.find_elements(By.CSS_SELECTOR,'div.tab-clients--title.simple > span')]
    except NoSuchElementException:
        clients=''

    #Awards
    try:
        awards_text=[k.text for k in driver.find_elements(By.CSS_SELECTOR,'div.tab-award--title')]
    except NoSuchElementException:
        awards_text=''


    try:
        awards_image=[k.get_attribute('src') for k in driver.find_elements(By.CSS_SELECTOR,'div.tab-award--image img')]
        awards_image_list=[]
        pgg=0
        for g in awards_image:
            pgg+=1
            response = requests.get(g)
            image = Image.open(BytesIO(response.content))
            awards_image_path=f'gallery//images//{uid}//awards//{title}_awards_pic_{pgg}.png'
            awards_image_path1=awards_image_path.replace("//",'/')
            directory = os.path.dirname(awards_image_path)
            if not os.path.exists(directory):
                os.makedirs(directory)
            with open(awards_image_path, "wb") as f:
                f.write(response.content)
            awards_image_list.append(awards_image_path1)

    except NoSuchElementException:
        awards_image_list=''

    awards_image_dict_list=[{
        'awards-text':awards_text,
        'awards-image':awards_image_list
    }]
    awards_image_json_list = json.dumps(awards_image_dict_list, indent=4)
    awards_image_json_list = json.loads(awards_image_json_list)


    #press mentions
    try:
        press_title=[k.text for k in driver.find_elements(By.CSS_SELECTOR,'span.press-source')]
        press_url=[k.get_attribute('href') for k in driver.find_elements(By.CSS_SELECTOR,'a.press-item')]
        press_desc=[k.text for k in driver.find_elements(By.CSS_SELECTOR,'p.press-title')]

        press_json_list=[]
        for f,g,h in zip(press_title,press_url,press_desc):
                press_dict={
                    'press-title':f,
                    'press-url':g,
                    'press-description':h,
                }
                press_json_list.append(press_dict)
    except NoSuchElementException:
        press_json_list=''

    #industries
    try:
        industries=[k.text for k in driver.find_elements(By.CLASS_NAME,'industries__wrap')][0].split("\n")
        industries_list=[]
        for k in industries:
            industries_dict={
                'industry-type':k,
                'industry-percent':'0%'
            }
            industries_list.append(industries_dict)
    except (IndexError, NoSuchElementException):
        industries=''

    #json_list
    new_data = {
    "uid": uid,
    "url":url,
    "website-url": contacts,
    "title": title,
    "profile-pic": logo_path1,
    "profile-slogan": profile_slogan,
    'star-rating':rating,
    'profile-overview':content,
    "full-address": full_address,
    "headquarters":headquarters,
    "country":country,
    "state":state,
    "city":city,
    'area':area,
    'headquarters-map':headquarters_map, #
    'other-locations':other_loc_json,
    "social-media-facebook": facebook,
    "social-media-twitter": twitter,
    "social-media-linkedin": linkedin,
    "social-media-instagram": instagram,
    "social-media-youtube": youtube,
    "no-of-employees": number_of_employees,
    "minimal-budget": minimal_budget, #
    "average-hourly-rate": average_hourly_rate,
    "year-founded": year_founded,
    'thumbnail-image-path':thumbnail_saved_path1,
    'video-urls':video_urls,
    'services':service_list_list,
    'industries':industries_list,
    'team-bios':team_bio_json_list,
    'clients':clients,
    'awards':awards_image_json_list,
    'press':press_json_list
    }

    #Append JSON to List
    json_list.append(new_data)
    print(new_data)
    
# Convert the dictionary to JSON format
json_data = json.dumps(json_list, indent=4)
print(json_data)

driver.quit()

{'uid': 8880000001, 'url': 'https://www.designrush.com/agency/profile/naked-development', 'website-url': 'https://apps.nakeddev.com/designrush', 'title': 'Naked Development', 'profile-pic': 'gallery/images/8880000001/logo/Naked Development.png', 'profile-slogan': 'We are a Creative Development Agency & Consultancy.', 'star-rating': '4.9', 'profile-overview': 'First we listen and strip down your idea to reveal the core of your business objectives. This means designing a strategy that will best support the users total experience, end-to-end. Then our US-based dev team works with the creative team to code with precision. An app is like a rocket, built to fly, but is no good if it never launches to the world.', 'full-address': '8815 Research Dr, Irvine, California 92618, United States', 'headquarters': '8815 Research Dr, Irvine, California 92618, United States', 'country': 'United States', 'state': 'Texas', 'city': 'Austin', 'area': 'Congress Ave. STE', 'headquarters-map': 'https://www.goo

In [30]:
with open("dr_31mar/dr Sample.json", "w") as json_file:
    json_file.write(json_data)

In [10]:
industries
for 

['eCommerce',
 'Entertainment',
 'Higher Education',
 'Travel',
 'Beauty',
 'Restaurants',
 'Real Estate',
 'Automotive']

In [24]:
for i in range(0,18):
    print(json_list[i]['headquarters-map'])

https://www.google.com/maps/search/8815+Research+Dr%2C+Irvine%2C+California+92618%2C+United+States
https://www.google.com/maps/search/4388+Rue+Saint-Denis%2C+suite+200%2C+%23305%2C+Montreal%2C+H2J+2L1%2C+Canada
https://www.google.com/maps/search/4130+Cahuenga+Blvd%2C+Suite+115%2C+Toluca+Lake%2C+Los+Angeles%2C+California+91602%2C+United+States
https://www.google.com/maps/search/House+62%2C+Road%3A+03%2C+Block%3A+B%2C+Niketon%2C+Gulshan-1%2C+Dhaka%2C+1213%2C+Bangladesh
https://www.google.com/maps/search/1301+K+St+NW+Ste+300W%2C+Washington+DC%2C+Washington+DC+20005%2C+United+States
https://www.google.com/maps/search/13105+Northwest+Fwy%2C+Suite+765%2C+Houston%2C+Texas+77040%2C+United+States
https://www.google.com/maps/search/541+NE+20th+Ave%2C+Suite+115%2C+Portland%2C+Oregon+97232%2C+United+States
https://www.google.com/maps/search/1983+Marcus+avenue+suite+204%2C+New+Hyde+Park%2C+New+York+11042%2C+United+States
https://www.google.com/maps/search/Grazitti+Interactive+Inc.%2C+Mountain+View%

In [21]:
'https://www.google.com/maps/search/8815+Research+Dr%2C+Irvine%2C+California+92618%2C+United+States'.split("%")

['https://www.google.com/maps/search/8815+Research+Dr',
 '2C+Irvine',
 '2C+California+92618',
 '2C+United+States']

In [28]:
from urllib.parse import urlparse, parse_qs

url = "https://www.google.com/maps/search/8815+Research+Dr%2C+Irvine%2C+California+92618%2C+United+States"

# Parse the URL
parsed_url = urlparse(url)

# Get the query part of the URL
query = parsed_url.query

# Parse the query string into a dictionary
query_dict = parse_qs(query)

# Get the address part from the dictionary
address = query_dict.get('q', [''])[0]

# Split the address by commas
parts = address.split(',')

# Extract the area, city, state, and country
area = parts[0].strip()
city = parts[-3].strip()
state = parts[-2].strip()
country = parts[-1].strip()

print("Area:", area)
print("City:", city)
print("State:", state)
print("Country:", country)


IndexError: list index out of range

In [26]:
for url in urls:
    # Extract the address part from the URL
    address = re.search(r'/search/(.*)', url).group(1)

    # Split the address by commas
    parts = address.split(',')

    # Extract the area, city, state, and country
    area = parts[0].strip()
    city = parts[-3].strip()
    state = parts[-2].strip()
    country = parts[-1].strip()

    print("URL:", url)
    print("Area:", area)
    print("City:", city)
    print("State:", state)
    print("Country:", country)
    print()

IndexError: list index out of range